In [28]:
import pandas as pd

df = pd.read_csv('vehicles_us.csv')

print(df)

df.info()



       price  model_year           model  condition  cylinders fuel  odometer  \
0       9400      2011.0          bmw x5       good        6.0  gas  145000.0   
1      25500         NaN      ford f-150       good        6.0  gas   88705.0   
2       5500      2013.0  hyundai sonata   like new        4.0  gas  110000.0   
3       1500      2003.0      ford f-150       fair        8.0  gas       NaN   
4      14900      2017.0    chrysler 200  excellent        4.0  gas   80903.0   
...      ...         ...             ...        ...        ...  ...       ...   
51520   9249      2013.0   nissan maxima   like new        6.0  gas   88136.0   
51521   2700      2002.0     honda civic    salvage        4.0  gas  181500.0   
51522   3950      2009.0  hyundai sonata  excellent        4.0  gas  128000.0   
51523   7455      2013.0  toyota corolla       good        4.0  gas  139573.0   
51524   6300      2014.0   nissan altima       good        4.0  gas       NaN   

      transmission    type 

In [29]:
print (df['model_year'].isna().sum())

3619


In [30]:
df['cylinders'].fillna(0)

0        6.0
1        6.0
2        4.0
3        8.0
4        4.0
        ... 
51520    6.0
51521    4.0
51522    4.0
51523    4.0
51524    4.0
Name: cylinders, Length: 51525, dtype: float64

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51525 entries, 0 to 51524
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         51525 non-null  int64  
 1   model_year    47906 non-null  float64
 2   model         51525 non-null  object 
 3   condition     51525 non-null  object 
 4   cylinders     46265 non-null  float64
 5   fuel          51525 non-null  object 
 6   odometer      43633 non-null  float64
 7   transmission  51525 non-null  object 
 8   type          51525 non-null  object 
 9   paint_color   42258 non-null  object 
 10  is_4wd        25572 non-null  float64
 11  date_posted   51525 non-null  object 
 12  days_listed   51525 non-null  int64  
dtypes: float64(4), int64(2), object(7)
memory usage: 5.1+ MB


In [32]:
print (df['cylinders'].isna().sum())

5260


In [33]:
df['cylinders'].fillna(0)


0        6.0
1        6.0
2        4.0
3        8.0
4        4.0
        ... 
51520    6.0
51521    4.0
51522    4.0
51523    4.0
51524    4.0
Name: cylinders, Length: 51525, dtype: float64

In [34]:
print(df['cylinders'].isna().sum())

5260


In [35]:
print(df['cylinders'])

0        6.0
1        6.0
2        4.0
3        8.0
4        4.0
        ... 
51520    6.0
51521    4.0
51522    4.0
51523    4.0
51524    4.0
Name: cylinders, Length: 51525, dtype: float64


In [36]:
print(df.cylinders)

0        6.0
1        6.0
2        4.0
3        8.0
4        4.0
        ... 
51520    6.0
51521    4.0
51522    4.0
51523    4.0
51524    4.0
Name: cylinders, Length: 51525, dtype: float64


In [37]:
display(df.isnull().sum())

price               0
model_year       3619
model               0
condition           0
cylinders        5260
fuel                0
odometer         7892
transmission        0
type                0
paint_color      9267
is_4wd          25953
date_posted         0
days_listed         0
dtype: int64

In [38]:
df = pd.read_csv('vehicles_us.csv')

print(df)

       price  model_year           model  condition  cylinders fuel  odometer  \
0       9400      2011.0          bmw x5       good        6.0  gas  145000.0   
1      25500         NaN      ford f-150       good        6.0  gas   88705.0   
2       5500      2013.0  hyundai sonata   like new        4.0  gas  110000.0   
3       1500      2003.0      ford f-150       fair        8.0  gas       NaN   
4      14900      2017.0    chrysler 200  excellent        4.0  gas   80903.0   
...      ...         ...             ...        ...        ...  ...       ...   
51520   9249      2013.0   nissan maxima   like new        6.0  gas   88136.0   
51521   2700      2002.0     honda civic    salvage        4.0  gas  181500.0   
51522   3950      2009.0  hyundai sonata  excellent        4.0  gas  128000.0   
51523   7455      2013.0  toyota corolla       good        4.0  gas  139573.0   
51524   6300      2014.0   nissan altima       good        4.0  gas       NaN   

      transmission    type 

In [39]:
df.columns

Index(['price', 'model_year', 'model', 'condition', 'cylinders', 'fuel',
       'odometer', 'transmission', 'type', 'paint_color', 'is_4wd',
       'date_posted', 'days_listed'],
      dtype='object')

In [40]:
df.isnull()


,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed
0,False,False,False,False,False,False,False,False,False,True,False,False,False
1,False,True,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,True,False,False
3,False,False,False,False,False,False,True,False,False,True,True,False,False
4,False,False,False,False,False,False,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
51520,False,False,False,False,False,False,False,False,False,False,True,False,False
51521,False,False,False,False,False,False,False,False,False,False,True,False,False
51522,False,False,False,False,False,False,False,False,False,False,True,False,False
51523,False,False,False,False,False,False,False,False,False,False,True,False,False


In [41]:
df[df.model_year.isnull()==True]


,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed
1,25500,NaN,ford f-150,good,6.0,gas,88705.0,automatic,pickup,white,1.0,2018-10-19,50
20,6990,NaN,chevrolet tahoe,excellent,8.0,gas,147485.0,automatic,SUV,silver,1.0,2018-08-05,28
65,12800,NaN,ford f-150,excellent,6.0,gas,108500.0,automatic,pickup,white,NaN,2018-09-23,15
69,7800,NaN,ford f-150,like new,8.0,gas,97510.0,automatic,truck,white,1.0,2019-02-20,39
72,3650,NaN,subaru impreza,excellent,NaN,gas,74000.0,automatic,sedan,blue,1.0,2018-08-07,60
...,...,...,...,...,...,...,...,...,...,...,...,...,...
51464,8890,NaN,ford f250 super duty,good,8.0,gas,135778.0,automatic,truck,red,1.0,2018-11-28,24
51465,34595,NaN,gmc yukon,excellent,8.0,gas,86456.0,automatic,SUV,silver,1.0,2019-04-18,13
51487,1495,NaN,ford f-150,fair,6.0,gas,15804.0,manual,truck,white,1.0,2018-09-18,56
51488,21900,NaN,chevrolet silverado 3500hd,excellent,8.0,diesel,203000.0,automatic,truck,white,1.0,2018-12-27,37


In [42]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51525 entries, 0 to 51524
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         51525 non-null  int64  
 1   model_year    47906 non-null  float64
 2   model         51525 non-null  object 
 3   condition     51525 non-null  object 
 4   cylinders     46265 non-null  float64
 5   fuel          51525 non-null  object 
 6   odometer      43633 non-null  float64
 7   transmission  51525 non-null  object 
 8   type          51525 non-null  object 
 9   paint_color   42258 non-null  object 
 10  is_4wd        25572 non-null  float64
 11  date_posted   51525 non-null  object 
 12  days_listed   51525 non-null  int64  
dtypes: float64(4), int64(2), object(7)
memory usage: 5.1+ MB


In [43]:
df.isnull().sum()


price               0
model_year       3619
model               0
condition           0
cylinders        5260
fuel                0
odometer         7892
transmission        0
type                0
paint_color      9267
is_4wd          25953
date_posted         0
days_listed         0
dtype: int64

In [44]:
columns_to_replace = ['model_year', 'cylinders', 'odometer', 'paint_color', 'is_4wd']
for col in columns_to_replace:
    df[col] = df[col].fillna('unknown')
    print(df)

       price model_year           model  condition  cylinders fuel  odometer  \
0       9400     2011.0          bmw x5       good        6.0  gas  145000.0   
1      25500    unknown      ford f-150       good        6.0  gas   88705.0   
2       5500     2013.0  hyundai sonata   like new        4.0  gas  110000.0   
3       1500     2003.0      ford f-150       fair        8.0  gas       NaN   
4      14900     2017.0    chrysler 200  excellent        4.0  gas   80903.0   
...      ...        ...             ...        ...        ...  ...       ...   
51520   9249     2013.0   nissan maxima   like new        6.0  gas   88136.0   
51521   2700     2002.0     honda civic    salvage        4.0  gas  181500.0   
51522   3950     2009.0  hyundai sonata  excellent        4.0  gas  128000.0   
51523   7455     2013.0  toyota corolla       good        4.0  gas  139573.0   
51524   6300     2014.0   nissan altima       good        4.0  gas       NaN   

      transmission    type paint_color 

In [45]:
df.isnull().sum()


price           0
model_year      0
model           0
condition       0
cylinders       0
fuel            0
odometer        0
transmission    0
type            0
paint_color     0
is_4wd          0
date_posted     0
days_listed     0
dtype: int64

In [46]:
df.duplicated()


0        False
1        False
2        False
3        False
4        False
         ...  
51520    False
51521    False
51522    False
51523    False
51524    False
Length: 51525, dtype: bool

In [47]:
df[df.duplicated()]


,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed


In [48]:
df.duplicated().sum()


0

In [49]:
df['model']


0                bmw x5
1            ford f-150
2        hyundai sonata
3            ford f-150
4          chrysler 200
              ...      
51520     nissan maxima
51521       honda civic
51522    hyundai sonata
51523    toyota corolla
51524     nissan altima
Name: model, Length: 51525, dtype: object

In [50]:
df.model.nunique()


100

In [51]:
df.model.unique()


array(['bmw x5', 'ford f-150', 'hyundai sonata', 'chrysler 200',
       'chrysler 300', 'toyota camry', 'honda pilot', 'kia sorento',
       'chevrolet silverado 1500', 'honda accord', 'ram 1500',
       'gmc yukon', 'jeep cherokee', 'chevrolet traverse',
       'hyundai elantra', 'chevrolet tahoe', 'toyota rav4',
       'chevrolet silverado', 'jeep wrangler', 'chevrolet malibu',
       'ford fusion se', 'chevrolet impala', 'chevrolet corvette',
       'jeep liberty', 'toyota camry le', 'nissan altima',
       'subaru outback', 'toyota highlander', 'dodge charger',
       'toyota tacoma', 'chevrolet equinox', 'nissan rogue',
       'mercedes-benz benze sprinter 2500', 'honda cr-v',
       'jeep grand cherokee', 'toyota 4runner', 'ford focus',
       'honda civic', 'kia soul', 'chevrolet colorado',
       'ford f150 supercrew cab xlt', 'chevrolet camaro lt coupe 2d',
       'chevrolet cruze', 'ford mustang', 'chevrolet silverado 3500hd',
       'nissan frontier crew cab sv', 'subaru imp

In [52]:
sorted(df.model.unique())


['acura tl',
 'bmw x5',
 'buick enclave',
 'cadillac escalade',
 'chevrolet camaro',
 'chevrolet camaro lt coupe 2d',
 'chevrolet colorado',
 'chevrolet corvette',
 'chevrolet cruze',
 'chevrolet equinox',
 'chevrolet impala',
 'chevrolet malibu',
 'chevrolet silverado',
 'chevrolet silverado 1500',
 'chevrolet silverado 1500 crew',
 'chevrolet silverado 2500hd',
 'chevrolet silverado 3500hd',
 'chevrolet suburban',
 'chevrolet tahoe',
 'chevrolet trailblazer',
 'chevrolet traverse',
 'chrysler 200',
 'chrysler 300',
 'chrysler town & country',
 'dodge charger',
 'dodge dakota',
 'dodge grand caravan',
 'ford econoline',
 'ford edge',
 'ford escape',
 'ford expedition',
 'ford explorer',
 'ford f-150',
 'ford f-250',
 'ford f-250 sd',
 'ford f-250 super duty',
 'ford f-350 sd',
 'ford f150',
 'ford f150 supercrew cab xlt',
 'ford f250',
 'ford f250 super duty',
 'ford f350',
 'ford f350 super duty',
 'ford focus',
 'ford focus se',
 'ford fusion',
 'ford fusion se',
 'ford mustang',
 '

In [53]:
df.condition.unique()


array(['good', 'like new', 'fair', 'excellent', 'salvage', 'new'],
      dtype=object)

In [54]:
df.type.unique()


array(['SUV', 'pickup', 'sedan', 'truck', 'coupe', 'van', 'convertible',
       'hatchback', 'wagon', 'mini-van', 'other', 'offroad', 'bus'],
      dtype=object)

In [55]:
df.paint_color.unique()


array(['unknown', 'white', 'red', 'black', 'blue', 'grey', 'silver',
       'custom', 'orange', 'yellow', 'brown', 'green', 'purple'],
      dtype=object)

In [56]:
df.fuel.unique()


array(['gas', 'diesel', 'other', 'hybrid', 'electric'], dtype=object)

In [57]:
df.transmission.unique()


array(['automatic', 'manual', 'other'], dtype=object)

In [58]:
df.describe()


,price,days_listed
count,51525.000000,51525.00000
mean,12132.464920,39.55476
std,10040.803015,28.20427
min,1.000000,0.00000
25%,5000.000000,19.00000
50%,9000.000000,33.00000
75%,16839.000000,53.00000
max,375000.000000,271.00000


In [59]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51525 entries, 0 to 51524
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   price         51525 non-null  int64 
 1   model_year    51525 non-null  object
 2   model         51525 non-null  object
 3   condition     51525 non-null  object
 4   cylinders     51525 non-null  object
 5   fuel          51525 non-null  object
 6   odometer      51525 non-null  object
 7   transmission  51525 non-null  object
 8   type          51525 non-null  object
 9   paint_color   51525 non-null  object
 10  is_4wd        51525 non-null  object
 11  date_posted   51525 non-null  object
 12  days_listed   51525 non-null  int64 
dtypes: int64(2), object(11)
memory usage: 5.1+ MB


In [60]:
df['paint_color'].mode()


0    white
Name: paint_color, dtype: object

In [61]:
df['model_year'].mode()


0    unknown
Name: model_year, dtype: object

In [62]:
df['transmission'].mode()


0    automatic
Name: transmission, dtype: object

In [63]:
df['fuel'].mode()


0    gas
Name: fuel, dtype: object

In [64]:
df['condition'].mode()


0    excellent
Name: condition, dtype: object

In [65]:
df['type'].mode()


0    SUV
Name: type, dtype: object

In [66]:
df['price'].max()


375000

In [67]:
df['price'].min()


1

In [68]:
df.sort_values(['price'], ascending = True)


,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed
22900,1,unknown,dodge charger,excellent,10.0,gas,25146.0,other,sedan,black,1.0,2018-09-29,60
10873,1,2018.0,ram 3500,excellent,6.0,gas,3047.0,automatic,truck,white,1.0,2018-11-22,27
9536,1,2016.0,dodge charger,excellent,10.0,gas,29406.0,other,sedan,red,1.0,2018-11-14,27
9537,1,unknown,ford fusion,excellent,4.0,gas,unknown,automatic,sedan,blue,1.0,2018-06-18,5
9538,1,2015.0,chevrolet camaro,excellent,10.0,gas,28926.0,other,coupe,grey,1.0,2018-10-06,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...
30634,189000,2014.0,ford f-150,good,6.0,gas,unknown,automatic,truck,black,unknown,2018-07-21,42
33434,189000,2014.0,ford f-150,good,6.0,gas,151248.0,automatic,truck,black,unknown,2019-02-05,102
27375,189000,2014.0,ford f-150,good,6.0,gas,151248.0,automatic,truck,black,unknown,2018-09-25,72
11359,300000,2015.0,ram 2500,excellent,unknown,diesel,unknown,automatic,truck,grey,1.0,2018-10-15,39


In [69]:
import pandas as pd

df = pd.read_csv('vehicles_us.csv')

print(df)

df.info()


       price  model_year           model  condition  cylinders fuel  odometer  \
0       9400      2011.0          bmw x5       good        6.0  gas  145000.0   
1      25500         NaN      ford f-150       good        6.0  gas   88705.0   
2       5500      2013.0  hyundai sonata   like new        4.0  gas  110000.0   
3       1500      2003.0      ford f-150       fair        8.0  gas       NaN   
4      14900      2017.0    chrysler 200  excellent        4.0  gas   80903.0   
...      ...         ...             ...        ...        ...  ...       ...   
51520   9249      2013.0   nissan maxima   like new        6.0  gas   88136.0   
51521   2700      2002.0     honda civic    salvage        4.0  gas  181500.0   
51522   3950      2009.0  hyundai sonata  excellent        4.0  gas  128000.0   
51523   7455      2013.0  toyota corolla       good        4.0  gas  139573.0   
51524   6300      2014.0   nissan altima       good        4.0  gas       NaN   

      transmission    type 

In [70]:
print(df['price'].value_counts()[1])


798


In [71]:
df['price'].describe()


count     51525.000000
mean      12132.464920
std       10040.803015
min           1.000000
25%        5000.000000
50%        9000.000000
75%       16839.000000
max      375000.000000
Name: price, dtype: float64

In [72]:
print(df.model_year.to_string(index=False))


2011.0
   NaN
2013.0
2003.0
2017.0
2014.0
2015.0
2013.0
2012.0
2008.0
2011.0
2012.0
2012.0
2018.0
2009.0
2013.0
2010.0
2014.0
2013.0
2018.0
   NaN
2007.0
2004.0
2004.0
2009.0
2017.0
2014.0
2015.0
2012.0
2010.0
2009.0
2017.0
2015.0
2011.0
2008.0
2015.0
2013.0
2005.0
2015.0
2009.0
2017.0
2017.0
2013.0
2008.0
2004.0
2001.0
2015.0
2008.0
2014.0
2006.0
2006.0
2012.0
2011.0
2012.0
2017.0
2017.0
2006.0
2012.0
2004.0
2006.0
2005.0
2013.0
2017.0
1966.0
2017.0
   NaN
2013.0
2012.0
2013.0
   NaN
2006.0
2018.0
   NaN
1994.0
2008.0
2005.0
2010.0
2012.0
2019.0
2012.0
2014.0
2000.0
2014.0
2008.0
   NaN
2000.0
2011.0
2009.0
2017.0
2015.0
2010.0
2010.0
2003.0
2017.0
2012.0
2017.0
2013.0
2012.0
2008.0
2018.0
2015.0
2017.0
   NaN
2004.0
2014.0
2015.0
2019.0
2017.0
2018.0
2013.0
2017.0
2018.0
2019.0
2014.0
   NaN
2018.0
   NaN
2012.0
2017.0
2016.0
2018.0
2006.0
2012.0
2015.0
2018.0
2019.0
2012.0
2010.0
2000.0
2005.0
2000.0
2009.0
2011.0
2018.0
2012.0
2016.0
2015.0
2014.0
2013.0
2014.0
1993.0
2010.0
2015.0

In [74]:
df['model_year'] = df['model_year'].fillna(0)
print(df['model_year'])


0        2011.0
1           0.0
2        2013.0
3        2003.0
4        2017.0
          ...  
51520    2013.0
51521    2002.0
51522    2009.0
51523    2013.0
51524    2014.0
Name: model_year, Length: 51525, dtype: float64


In [75]:
df['model_year'] = df['model_year'].astype(int)
display(df.dtypes)


price             int64
model_year        int64
model            object
condition        object
cylinders       float64
fuel             object
odometer        float64
transmission     object
type             object
paint_color      object
is_4wd          float64
date_posted      object
days_listed       int64
dtype: object

In [77]:
convert_to_int = lambda x: int(x) if x % 1 == 0 else x
data_int = df['model_year'].apply(convert_to_int)
data_int = data_int.astype(dtype=int)
print(data_int)


0        2011
1           0
2        2013
3        2003
4        2017
         ... 
51520    2013
51521    2002
51522    2009
51523    2013
51524    2014
Name: model_year, Length: 51525, dtype: int64


In [78]:
print(df)


       price  model_year           model  condition  cylinders fuel  odometer  \
0       9400        2011          bmw x5       good        6.0  gas  145000.0   
1      25500           0      ford f-150       good        6.0  gas   88705.0   
2       5500        2013  hyundai sonata   like new        4.0  gas  110000.0   
3       1500        2003      ford f-150       fair        8.0  gas       NaN   
4      14900        2017    chrysler 200  excellent        4.0  gas   80903.0   
...      ...         ...             ...        ...        ...  ...       ...   
51520   9249        2013   nissan maxima   like new        6.0  gas   88136.0   
51521   2700        2002     honda civic    salvage        4.0  gas  181500.0   
51522   3950        2009  hyundai sonata  excellent        4.0  gas  128000.0   
51523   7455        2013  toyota corolla       good        4.0  gas  139573.0   
51524   6300        2014   nissan altima       good        4.0  gas       NaN   

      transmission    type 

In [79]:
df['odometer'] = df['odometer'].fillna(0)
print(df['odometer'])



0        145000.0
1         88705.0
2        110000.0
3             0.0
4         80903.0
           ...   
51520     88136.0
51521    181500.0
51522    128000.0
51523    139573.0
51524         0.0
Name: odometer, Length: 51525, dtype: float64


In [80]:
convert_to_int = lambda x: int(x) if x % 1 == 0 else x
data_int = df['odometer'].apply(convert_to_int)
data_int = data_int.astype(dtype=int)
print(data_int)

0        145000
1         88705
2        110000
3             0
4         80903
          ...  
51520     88136
51521    181500
51522    128000
51523    139573
51524         0
Name: odometer, Length: 51525, dtype: int64
